In [30]:
# Importation of modules
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage import filters
from time import perf_counter
import os
from cropping import masking
from pprint import pprint
import time
%matplotlib inline

In [32]:
# Create a dictionary location:value --> {'270-01-01': 's_2-s_7-s_0-s_pt',...}

def get_transcription(transcription_file):
    with open(transcription_file, 'r') as f:
        names = {}
        for line in f.readlines():
            names[line.split(" ")[0]] = line.split(" ")[1].strip()
    return names

def get_list_of_text(file):
    with open(file, 'r') as f:
        nameOfText = []
        for line in f.readlines():
            nameOfText.append(line.strip())
    return nameOfText

#transciptionOfAllWords = get_transcription("./../ground-truth/transcription.txt")
trainingFiles = get_list_of_text("./../task/train.txt")
testFiles = get_list_of_text("./../task2/test.txt")
print(">> Dictionary and sets created")
print(trainingFiles)

>> Dictionary and sets created
['271', '272', '273', '274', '276', '300']


In [10]:
# Just do that one time to generate the .pkl files -> If files are already available skip this cell
def cropPictures():
    allPictures = os.listdir("./../images")

    if not "pkls" in os.listdir("."):
        os.makedirs("./pkls")
    
    allPictures = [val for val in allPictures if int(val.split(".")[0]) >= 304]

    for picture in allPictures:
        
        print(">> Cropping picture", picture[:-4], " ...")
        start_time = time.perf_counter()
        numberOfPicture = picture[:-4]

        # Get all split words
        trimmed = masking(int(numberOfPicture))
    
        # Serialize the objects with pickle --> creation of all .pkl files
        with open(os.path.join("./pkls", numberOfPicture), 'wb') as f:
            pickle.dump(trimmed, f, pickle.HIGHEST_PROTOCOL)
        print(" Picture %s cropped in: %ss" % (picture[:-4], int(time.perf_counter()-start_time)))
        
cropPictures()

>> Cropping picture 309  ...
 Picture 309 cropped in: 126s
>> Cropping picture 307  ...
 Picture 307 cropped in: 166s
>> Cropping picture 308  ...
 Picture 308 cropped in: 158s
>> Cropping picture 305  ...
 Picture 305 cropped in: 180s
>> Cropping picture 304  ...
 Picture 304 cropped in: 140s
>> Cropping picture 306  ...
 Picture 306 cropped in: 154s


In [33]:
def binarizeAndStoreInDict(typeOfSet):
    allPkls = os.listdir("./pkls")

    #if not "cropped" in os.listdir("."):
        #os.makedirs("./cropped")

    # Dictionary containing the identifier and the values of all pixels of each word
    allWordsAsNumpyMatrix={}
    height=0
    width=0
    
    # Get maximum width and maximum height of all words in order to create a picture of same size 
    # for all words of all pages
    for text in allPkls:
        with open('./pkls/'+text,'rb') as f:
            allWords = pickle.load(f)
            maxHeightPage = max([x.shape[0] for x in allWords.values()])
            maxWidthPage = max([x.shape[1] for x in allWords.values()])
            if maxHeightPage>height:
                height= maxHeightPage
            if maxWidthPage > width:
                width= maxWidthPage

    for text in allPkls:
        if text not in typeOfSet:
            continue
        with open('./pkls/'+text,'rb') as f:
            allWords = pickle.load(f)
            
            for wordpkl in allWords:
                word = allWords[wordpkl].filled(255)
            
                # Only keep the dark parts
                tresh = filters.threshold_otsu(word)
                foo = word<tresh
            
                # Set every "word" as a matrix of same size.
                target = np.zeros((height,width))
                target[:foo.shape[0],:foo.shape[1]]=foo
            
                # return a dictionary '304-01-01':[[0,0,0,0,0,0,1,...,0,..], [], [0,1,0,0,..], []..]
                allWordsAsNumpyMatrix[wordpkl]=target
                
    return allWordsAsNumpyMatrix
allWords_training = binarizeAndStoreInDict(trainingFiles)
allWords_test = binarizeAndStoreInDict(testFiles)

In [34]:
print(allWords_training)

{'272-02-01': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), '272-02-02': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), '272-02-03': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), '272-02-04': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0.,

In [35]:
# JUST A TEST TO SEE IF ARRAYS OF FEATURES ARE WELL CREATED
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# Transform each picture of word into an array of features and store it in a dictionary identifier: allfeatures
# --> {'304-01-01': [feature1, feature2, feature3, feature4, feature5]}
from features import *
allWords_training_features = calculate_features_of_all_samples(allWords_training)
print("tmt")
allWords_test_features = calculate_features_of_all_samples(allWords_test)





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
tmt


In [20]:
print(allWords_training_features)

{'270-01-01': array([[-0.86781578,  1.20095115, -0.86781578,  1.21380063, -0.85496629],
       [-0.86781578,  1.20095115, -0.86781578,  1.21380063, -0.85496629],
       [-0.86781578,  1.20095115, -0.86781578,  1.21380063, -0.85496629],
       ...,
       [-0.86781578,  1.20095115, -0.86781578,  1.21380063, -0.85496629],
       [-0.86781578,  1.20095115, -0.86781578,  1.21380063, -0.85496629],
       [-0.86781578,  1.20095115, -0.86781578,  1.21380063, -0.85496629]]), '270-01-02': array([[-0.91292094,  1.13985678, -0.91292094,  1.15260695, -0.90017077],
       [-0.91292094,  1.13985678, -0.91292094,  1.15260695, -0.90017077],
       [-0.91292094,  1.13985678, -0.91292094,  1.15260695, -0.90017077],
       ...,
       [-0.91292094,  1.13985678, -0.91292094,  1.15260695, -0.90017077],
       [-0.91292094,  1.13985678, -0.91292094,  1.15260695, -0.90017077],
       [-0.91292094,  1.13985678, -0.91292094,  1.15260695, -0.90017077]]), '270-01-03': array([[-0.90886223,  1.17963932, -0.9088622

In [21]:
print(allWords_test_features)

{'307-02-01': array([[-0.92135939,  1.14843528, -0.92135939,  1.16129114, -0.90850352],
       [-0.92135939,  1.14843528, -0.92135939,  1.16129114, -0.90850352],
       [-0.92135939,  1.14843528, -0.92135939,  1.16129114, -0.90850352],
       ...,
       [-0.92135939,  1.14843528, -0.92135939,  1.16129114, -0.90850352],
       [-0.92135939,  1.14843528, -0.92135939,  1.16129114, -0.90850352],
       [-0.92135939,  1.14843528, -0.92135939,  1.16129114, -0.90850352]]), '307-02-02': array([[-0.91663941,  1.17585558, -0.91663941,  1.18885245, -0.90364255],
       [-0.91663941,  1.17585558, -0.91663941,  1.18885245, -0.90364255],
       [-0.91663941,  1.17585558, -0.91663941,  1.18885245, -0.90364255],
       ...,
       [-0.91663941,  1.17585558, -0.91663941,  1.18885245, -0.90364255],
       [-0.91663941,  1.17585558, -0.91663941,  1.18885245, -0.90364255],
       [-0.91663941,  1.17585558, -0.91663941,  1.18885245, -0.90364255]]), '307-02-03': array([[-0.88482396,  1.20435488, -0.8848239

In [46]:
from dtw import *
from collections import defaultdict

with open("../task2/keywords.txt") as file:
    data = file.readlines()

res = defaultdict(list)

for testWord in data:
    word, id_ = testWord.strip().split(",")
    print(id_)
    for id_train, property_train in allWords_test_features.items():
        score = fastdtw(allWords_training_features[id_], property_train)
        res[word].append((id_train, score))
        print("indic")
print(res)

271-04-07
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic


indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indi

indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indic
indi

KeyboardInterrupt: 

In [47]:
print(res)

defaultdict(<class 'list'>, {'V-i-r-g-i-n-i-a': [('307-02-01', 54034.33339131091), ('307-02-02', 39075.351343351416), ('307-02-03', 74467.94512583522), ('307-02-04', 176088.91061551735), ('307-02-05', 42948.32365047776), ('307-02-06', 90833.61878342464), ('307-04-02', 101462.72843952189), ('307-04-03', 53205.99312001045), ('307-04-04', 86458.21981950756), ('307-04-05', 64715.99543871188), ('307-05-01', 121855.93686085912), ('307-05-02', 63291.932353039134), ('307-05-03', 92172.1569672297), ('307-05-04', 60945.87388297655), ('307-05-05', 92324.25961188332), ('307-05-06', 116934.12261977262), ('307-05-07', 109916.00513435305), ('307-06-01', 106647.27187315164), ('307-06-02', 73301.63780792711), ('307-06-03', 119679.24448709437), ('307-06-04', 89501.59320082027), ('307-06-05', 135752.3870477675), ('307-06-06', 61413.66900538669), ('307-06-07', 102108.8005372457), ('307-06-08', 119719.26553536588), ('307-06-09', 66296.07111880877), ('307-06-10', 109901.32464686061), ('307-07-01', 83421.884

In [53]:
print(res)

D-e-c-e-m-b-e-r, 307-02-01, 61182.68360362752, 307-02-02, 70084.08676800235, 307-02-03, 118148.28288058192, 307-02-04, 152705.47330018433, 307-02-05, 41465.06885556542, 307-02-06, 128976.7354865368, 307-04-02, 136022.42492380211, 307-04-03, 69914.77143324289, 307-04-04, 113422.18670455381, 307-04-05, 102966.94971176836, 307-05-01, 174293.52894458163, 307-05-02, 91926.53492890621, 307-05-03, 145952.7977688321, 307-05-04, 89605.3164427448, 307-05-05, 132999.98164478317, 307-05-06, 174563.4528378292, 307-05-07, 164999.69765689736, 307-06-01, 145804.14199835493, 307-06-02, 107852.61728930811, 307-06-03, 147299.08163981076, 307-06-04, 132464.79719356104, 307-06-05, 188550.45959204846, 307-06-06, 93756.06744684177, 307-06-07, 133613.59243383884, 307-06-08, 159096.56694224395, 307-06-09, 69815.43311267569, 307-06-10, 153201.26743715777, 307-07-01, 133805.8130519339, 307-09-01, 125771.54111318533, 307-09-02, 131712.0520585163, 307-09-03, 87363.29519270966, 307-09-04, 114377.10912023237, 307-09

In [51]:
with open("output.txt", "w") as file_:
    for word, results in res.items():
        res_str = ", ".join([f"{id_}, {val}" for id_, val in results])
        res = f"{word}, {res_str}\n"
        file_.write(res)

AttributeError: 'str' object has no attribute 'items'

In [12]:
#Just a test to compare the distance of an image with itself (should be equal to 0) and an image with another one
from dtw import *

start_time = time.perf_counter()
score1 = dtw(allWords_training_features['270-09-06'],allWords_training_features['270-05-07'])
score2 = dtw(allWords_training_features['270-09-06'],allWords_training_features['270-09-06'])
print(score1)
print(score2)
print("%ss" % (int(time.perf_counter()-start_time)))

NameError: name 'allWords_training_features' is not defined

In [30]:
#Just a test to compare the distance of an image with itself (should be equal to 0) and an image with another one
from dtw import *

start_time = time.perf_counter()
score1 = fastdtw(allWords_training_features['270-09-06'],allWords_training_features['270-05-07'])
score2 = fastdtw(allWords_training_features['270-09-06'],allWords_training_features['270-09-06'])
print(score1)
print(score2)
print("%ss" % (int(time.perf_counter()-start_time)))

13854.100135244295
0.0
7s


In [57]:
## IS TOO SLOW FOR NOW... NOT TESTED
import operator

def kNN_DTW(allWords_training, allWords_test, k, transcriptionOfAllWords):
# Returns the k nearest neighbors through DTW distance algorithm
    allWords_training_features = calculate_features_of_all_samples(allWords_training)
    allWords_test_features = calculate_features_of_all_samples(allWords_test)
    
    for test_image in allWords_test_features.keys():
        # compute distance between each training instance and the test instance
        # and store tuple of result and corresponding training instance
        currLabel = str(transciptionOfAllWords[test_image])
        labelOccureInTraing = 0
        distances = []
        for training_image in allWords_training_features.keys():
            dist = fastdtw(allWords_test_features[test_image], allWords_training_features[training_image])
            distances.append((transciptionOfAllWords[training_image], dist, training_image))
            if(currLabel == transciptionOfAllWords[training_image]):
                labelOccureInTraing += 1
                
        # sort list of tuples of distances in ascending order 
        # regarding the second item of the tuples, i.e. the distances
        distances.sort(key=operator.itemgetter(1))
        kNNforMajority = []
        locationOfWords = []
        #for x in range(k):
        #   kNNforMajority.append(distances[x][0]) 
        #   locationOfWords.append(distances[x][2])
        # return the k first training instances (the k most similar)
        #return majority(kNNforMajority), locationOfWords#
        print(currLabel + ":")
        arrPrecRec = eval(distances,k,currLabel,labelOccureInTraing)
        for x in range(k):
            print(str(distances[x][1]) + "," + str(distances[x][2])+ ", Match: " + str((distances[x][0]) == currLabel) + ", Precision: " + str(arrPrecRec[x][0]) + ", Recall: " + str(arrPrecRec[x][1]) +  ";" )
        
            
def eval(orderedDis,k,label,labelOccureInTraing):
    count = 0
    tp = 0
    arrPrecRec = []
    #theoretically before the first dataset the precision is 1 and recall is 0
    #arrPrecRec.append([1,0])
    for x in range(k):
        count += 1
        if label == orderedDis[x][0]:
            tp += 1
            
        #[Precision, Recall]
        arrPrecRec.append([tp/count,tp/labelOccureInTraing])
    return arrPrecRec
        
        
        
def majority(kNN):
# Returns the label occuring the most among the k nearest neighbors
    # classVotes is a key:value list
    print(kNN)
    classVotes = {}
    for x in range(len(kNN)):
        # extract the label of training instance 
        response = kNN[x][0]
        # if the label appears in list of classVotes 
        if response in classVotes:
        # add vote at to label key
            classVotes[response] += 1  
        else:
            classVotes[response] = 1
    # sort the classVotes according to the number of votes in a descending order
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

# test with one word
kNN_DTW(allWords_training, {'304-32-03':allWords_validation['304-32-03']},20, transciptionOfAllWords)



o-f:
6927.96655805848,270-08-03, Match: True, Precision: 1.0, Recall: 0.09090909090909091;
7203.095685180449,270-07-07, Match: True, Precision: 1.0, Recall: 0.18181818181818182;
7732.521188523189,270-07-01, Match: True, Precision: 1.0, Recall: 0.2727272727272727;
8911.414908689123,270-25-03, Match: True, Precision: 1.0, Recall: 0.36363636363636365;
9503.406374721944,270-19-09, Match: True, Precision: 1.0, Recall: 0.45454545454545453;
9790.48481795791,270-21-06, Match: True, Precision: 1.0, Recall: 0.5454545454545454;
9983.328247981437,270-23-05, Match: True, Precision: 1.0, Recall: 0.6363636363636364;
10864.572440214686,270-05-04, Match: True, Precision: 1.0, Recall: 0.7272727272727273;
10916.536344795277,270-10-02, Match: True, Precision: 1.0, Recall: 0.8181818181818182;
10959.142804449088,270-23-03, Match: False, Precision: 0.9, Recall: 0.8181818181818182;
10984.483872134115,270-33-03, Match: False, Precision: 0.8181818181818182, Recall: 0.8181818181818182;
12778.352196976335,270-05-